In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned/cleaned_data_100.csv


In [23]:
import tensorflow as tf
import numpy as np
from keras.layers import Embedding, Conv1D, MaxPooling1D
from keras.layers import Flatten, Dropout, Dense
import torch
from keras.models import Sequential
from torch.autograd import Variable
from torch import IntTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import re
import string
import itertools
import collections
import pandas as pd
from nltk import word_tokenize, pos_tag
import warnings
from collections import defaultdict, Counter
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from nltk.probability import FreqDist

In [3]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import pandas as pd
data = pd.read_csv("../input/cleaned/cleaned_data_100.csv")
data['X_data'] = data['X_data'].apply(lambda s: s.split(' ', 1)[1])


In [5]:
data

,X_data,Y_data
0,203518 143 INFO dfs DataNode DataXceiver Recei...,0
1,203518 35 INFO dfs FSNamesystem BLOCK NameSyst...,0
2,203519 143 INFO dfs DataNode DataXceiver Recei...,0
3,203519 145 INFO dfs DataNode DataXceiver Recei...,0
4,203519 145 INFO dfs DataNode PacketResponder P...,0
...,...,...
104810,210122 13 INFO dfs DataBlockScanner Verificati...,0
104811,210124 13 INFO dfs DataBlockScanner Verificati...,0
104812,210126 13 INFO dfs DataBlockScanner Verificati...,0
104813,210126 13 INFO dfs DataBlockScanner Verificati...,0


In [6]:
X = list(data['X_data'])

In [7]:
words_in_train = [sent.split() for sent in X] # Splitting the tweet into tokens
all_words_train = list(itertools.chain(*words_in_train)) # Making a flatlist of all the words in the corpus
all_words = all_words_train  
corpus = X  
vocab_size = len(set(all_words)) # number of words to keep.
embedding_dim = 100 # Dimension of the vector representation for each word 
max_length = 0
for i in range(len(corpus)):
    x = word_tokenize(corpus[i])
    if len(x) > max_length:
        max_length = len(x)

In [8]:
print("Max length of the sequence :",max_length)
print("Size of the vocabulary :",vocab_size)

Max length of the sequence : 46
Size of the vocabulary : 28048


In [9]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(data['X_data'])
word_index = tokenizer.word_index

In [10]:
# Pad the sequences so that they are all the same length
training_sequences = tokenizer.texts_to_sequences(data['X_data'])
trainds_vec = tf.keras.preprocessing.sequence.pad_sequences(training_sequences,maxlen=max_length, padding='post')

In [11]:
# import library
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(np.array(trainds_vec), data['Y_data'])
print('Original dataset shape', np.array(trainds_vec).shape)
print('Resample dataset shape', x_smote.shape)


Original dataset shape (104815, 46)
Resample dataset shape (203088, 46)


In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(x_smote,y_smote,test_size=0.2)

In [13]:
Training_labels = tf.keras.utils.to_categorical(Y_train)
Testing_labels = tf.keras.utils.to_categorical(Y_test)

In [14]:
X_train1,X_test1,Y_train1,Y_test1 = train_test_split(np.array(trainds_vec),data['Y_data'],test_size=0.2)
Training_labels1 = tf.keras.utils.to_categorical(Y_train1)
Testing_labels1 = tf.keras.utils.to_categorical(Y_test1)

In [15]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

2022-11-06 08:27:44.822571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 08:27:44.823482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 08:27:44.824475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 08:27:44.825298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 08:27:44.826000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# BILSTM

## with smote

In [25]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 10, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 100)               46000     
_________________________________________________________________
dense_6 (Dense)              (None, 32)                3232      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)               

In [26]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 10
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
1270/1270 [==============================] - 69s 54ms/step - loss: 0.1844 - accuracy: 0.9289 - precision: 0.9289 - recall: 0.9289 - auc: 0.9763 - val_loss: 0.1241 - val_accuracy: 0.9586 - val_precision: 0.9586 - val_recall: 0.9586 - val_auc: 0.9899
Epoch 2/5
1270/1270 [==============================] - 68s 54ms/step - loss: 0.1125 - accuracy: 0.9631 - precision: 0.9631 - recall: 0.9631 - auc: 0.9909 - val_loss: 0.1176 - val_accuracy: 0.9602 - val_precision: 0.9602 - val_recall: 0.9602 - val_auc: 0.9907
Epoch 3/5
1270/1270 [==============================] - 66s 52ms/step - loss: 0.1082 - accuracy: 0.9639 - precision: 0.9639 - recall: 0.9639 - auc: 0.9916 - val_loss: 0.1393 - val_accuracy: 0.9542 - val_precision: 0.9542 - val_recall: 0.9542 - val_auc: 0.9868
Epoch 4/5
1270/1270 [==============================] - 68s 54ms/step - loss: 0.1023 - accuracy: 0.9652 - precision: 0.9652 - recall: 0.9652 - auc: 0.9925 - val_loss: 0.1206 - val_accuracy: 0.9619 - val_precision: 0.9619 - v

In [27]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]

print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[19885   203]
 [ 1274 19256]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     20088
           1       0.99      0.94      0.96     20530

    accuracy                           0.96     40618
   macro avg       0.96      0.96      0.96     40618
weighted avg       0.96      0.96      0.96     40618



## without smote

In [31]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 10, 64)            0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100)               46000     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                3232      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)               

In [32]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/5
656/656 [==============================] - 34s 52ms/step - loss: 0.0712 - accuracy: 0.9856 - precision: 0.9856 - recall: 0.9856 - auc: 0.9955 - val_loss: 0.0118 - val_accuracy: 0.9977 - val_precision: 0.9977 - val_recall: 0.9977 - val_auc: 0.9991
Epoch 2/5
656/656 [==============================] - 35s 54ms/step - loss: 0.0076 - accuracy: 0.9986 - precision: 0.9986 - recall: 0.9986 - auc: 0.9994 - val_loss: 0.0070 - val_accuracy: 0.9988 - val_precision: 0.9988 - val_recall: 0.9988 - val_auc: 0.9994
Epoch 3/5
656/656 [==============================] - 35s 54ms/step - loss: 0.0040 - accuracy: 0.9993 - precision: 0.9993 - recall: 0.9993 - auc: 0.9997 - val_loss: 0.0111 - val_accuracy: 0.9983 - val_precision: 0.9983 - val_recall: 0.9983 - val_auc: 0.9993
Epoch 4/5
656/656 [==============================] - 34s 52ms/step - loss: 0.0034 - accuracy: 0.9992 - precision: 0.9992 - recall: 0.9992 - auc: 0.9999 - val_loss: 0.0094 - val_accuracy: 0.9985 - val_precision: 0.9985 - val_recal

In [34]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))


[[20277    33]
 [   20   633]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20310
           1       0.95      0.97      0.96       653

    accuracy                           1.00     20963
   macro avg       0.97      0.98      0.98     20963
weighted avg       1.00      1.00      1.00     20963



# LSTM

## With smote

In [35]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 10, 64)            0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                23000     
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1632      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)               

In [36]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
1270/1270 [==============================] - 59s 46ms/step - loss: 0.1939 - accuracy: 0.9335 - precision: 0.9335 - recall: 0.9335 - auc: 0.9812 - val_loss: 0.1218 - val_accuracy: 0.9594 - val_precision: 0.9594 - val_recall: 0.9594 - val_auc: 0.9901
Epoch 2/5
1270/1270 [==============================] - 55s 43ms/step - loss: 0.1060 - accuracy: 0.9640 - precision: 0.9640 - recall: 0.9640 - auc: 0.9920 - val_loss: 0.1041 - val_accuracy: 0.9644 - val_precision: 0.9644 - val_recall: 0.9644 - val_auc: 0.9927
Epoch 3/5
1270/1270 [==============================] - 58s 45ms/step - loss: 0.0892 - accuracy: 0.9690 - precision: 0.9690 - recall: 0.9690 - auc: 0.9943 - val_loss: 0.1073 - val_accuracy: 0.9645 - val_precision: 0.9645 - val_recall: 0.9645 - val_auc: 0.9921
Epoch 4/5
1270/1270 [==============================] - 58s 46ms/step - loss: 0.0829 - accuracy: 0.9707 - precision: 0.9707 - recall: 0.9707 - auc: 0.9950 - val_loss: 0.1059 - val_accuracy: 0.9639 - val_precision: 0.9639 - v

In [37]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[19840   248]
 [ 1194 19336]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     20088
           1       0.99      0.94      0.96     20530

    accuracy                           0.96     40618
   macro avg       0.97      0.96      0.96     40618
weighted avg       0.97      0.96      0.96     40618



## Without smote

In [45]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                23000     
_________________________________________________________________
dense_20 (Dense)             (None, 32)                1632      
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)              

In [46]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/5
656/656 [==============================] - 28s 43ms/step - loss: 0.0750 - accuracy: 0.9729 - precision: 0.9729 - recall: 0.9729 - auc: 0.9929 - val_loss: 0.0187 - val_accuracy: 0.9960 - val_precision: 0.9960 - val_recall: 0.9960 - val_auc: 0.9988
Epoch 2/5
656/656 [==============================] - 30s 46ms/step - loss: 0.0159 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9964 - auc: 0.9995 - val_loss: 0.0159 - val_accuracy: 0.9965 - val_precision: 0.9965 - val_recall: 0.9965 - val_auc: 0.9989
Epoch 3/5
656/656 [==============================] - 28s 43ms/step - loss: 0.0106 - accuracy: 0.9979 - precision: 0.9979 - recall: 0.9979 - auc: 0.9997 - val_loss: 0.0161 - val_accuracy: 0.9971 - val_precision: 0.9971 - val_recall: 0.9971 - val_auc: 0.9990
Epoch 4/5
656/656 [==============================] - 28s 43ms/step - loss: 0.0094 - accuracy: 0.9980 - precision: 0.9980 - recall: 0.9980 - auc: 0.9998 - val_loss: 0.0169 - val_accuracy: 0.9971 - val_precision: 0.9971 - val_recal

In [47]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))

[[20285    25]
 [   17   636]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20310
           1       0.96      0.97      0.97       653

    accuracy                           1.00     20963
   macro avg       0.98      0.99      0.98     20963
weighted avg       1.00      1.00      1.00     20963



# GRU

## with smote

In [42]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(50),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 10, 64)            0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
gru (GRU)                    (None, 50)                17400     
_________________________________________________________________
dense_18 (Dense)             (None, 32)                1632      
_________________________________________________________________
dropout_19 (Dropout)         (None, 32)              

In [43]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
1270/1270 [==============================] - 56s 44ms/step - loss: 0.1954 - accuracy: 0.9485 - precision: 0.9485 - recall: 0.9485 - auc: 0.9807 - val_loss: 0.1226 - val_accuracy: 0.9590 - val_precision: 0.9590 - val_recall: 0.9590 - val_auc: 0.9898
Epoch 2/5
1270/1270 [==============================] - 56s 44ms/step - loss: 0.1190 - accuracy: 0.9604 - precision: 0.9604 - recall: 0.9604 - auc: 0.9901 - val_loss: 0.1187 - val_accuracy: 0.9622 - val_precision: 0.9622 - val_recall: 0.9622 - val_auc: 0.9905
Epoch 3/5
1270/1270 [==============================] - 57s 45ms/step - loss: 0.1116 - accuracy: 0.9629 - precision: 0.9629 - recall: 0.9629 - auc: 0.9911 - val_loss: 0.1145 - val_accuracy: 0.9631 - val_precision: 0.9631 - val_recall: 0.9631 - val_auc: 0.9910
Epoch 4/5
1270/1270 [==============================] - 54s 43ms/step - loss: 0.1055 - accuracy: 0.9654 - precision: 0.9654 - recall: 0.9654 - auc: 0.9919 - val_loss: 0.1178 - val_accuracy: 0.9627 - val_precision: 0.9627 - v

In [44]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[20019    69]
 [ 1686 18844]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     20088
           1       1.00      0.92      0.96     20530

    accuracy                           0.96     40618
   macro avg       0.96      0.96      0.96     40618
weighted avg       0.96      0.96      0.96     40618



## without smote 

In [48]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(50),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 50)                17400     
_________________________________________________________________
dense_22 (Dense)             (None, 32)                1632      
_________________________________________________________________
dropout_23 (Dropout)         (None, 32)              

In [49]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/5
656/656 [==============================] - 30s 45ms/step - loss: 0.0657 - accuracy: 0.9874 - precision: 0.9874 - recall: 0.9874 - auc: 0.9953 - val_loss: 0.0170 - val_accuracy: 0.9972 - val_precision: 0.9972 - val_recall: 0.9972 - val_auc: 0.9980
Epoch 2/5
656/656 [==============================] - 28s 42ms/step - loss: 0.0095 - accuracy: 0.9977 - precision: 0.9977 - recall: 0.9977 - auc: 0.9994 - val_loss: 0.0073 - val_accuracy: 0.9984 - val_precision: 0.9984 - val_recall: 0.9984 - val_auc: 0.9995
Epoch 3/5
656/656 [==============================] - 28s 42ms/step - loss: 0.0047 - accuracy: 0.9987 - precision: 0.9987 - recall: 0.9987 - auc: 0.9998 - val_loss: 0.0086 - val_accuracy: 0.9990 - val_precision: 0.9990 - val_recall: 0.9990 - val_auc: 0.9995
Epoch 4/5
656/656 [==============================] - 28s 42ms/step - loss: 0.0021 - accuracy: 0.9998 - precision: 0.9998 - recall: 0.9998 - auc: 0.9999 - val_loss: 0.0065 - val_accuracy: 0.9995 - val_precision: 0.9995 - val_recal

In [50]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))

[[20310     0]
 [   10   643]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20310
           1       1.00      0.98      0.99       653

    accuracy                           1.00     20963
   macro avg       1.00      0.99      1.00     20963
weighted avg       1.00      1.00      1.00     20963



# BIGRU

## With smote

In [51]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(50)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100)               34800     
_________________________________________________________________
dense_24 (Dense)             (None, 32)                3232      
_________________________________________________________________
dropout_25 (Dropout)         (None, 32)              

In [52]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
1270/1270 [==============================] - 65s 51ms/step - loss: 0.1796 - accuracy: 0.9410 - precision: 0.9410 - recall: 0.9410 - auc: 0.9838 - val_loss: 0.1194 - val_accuracy: 0.9602 - val_precision: 0.9602 - val_recall: 0.9602 - val_auc: 0.9912
Epoch 2/5
1270/1270 [==============================] - 63s 49ms/step - loss: 0.1121 - accuracy: 0.9626 - precision: 0.9626 - recall: 0.9626 - auc: 0.9911 - val_loss: 0.1203 - val_accuracy: 0.9595 - val_precision: 0.9595 - val_recall: 0.9595 - val_auc: 0.9903
Epoch 3/5
1270/1270 [==============================] - 63s 49ms/step - loss: 0.1066 - accuracy: 0.9637 - precision: 0.9637 - recall: 0.9637 - auc: 0.9919 - val_loss: 0.1108 - val_accuracy: 0.9642 - val_precision: 0.9642 - val_recall: 0.9642 - val_auc: 0.9911
Epoch 4/5
1270/1270 [==============================] - 62s 49ms/step - loss: 0.1024 - accuracy: 0.9652 - precision: 0.9652 - recall: 0.9652 - auc: 0.9925 - val_loss: 0.1118 - val_accuracy: 0.9615 - val_precision: 0.9615 - v

In [53]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[20000    88]
 [ 1778 18752]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     20088
           1       1.00      0.91      0.95     20530

    accuracy                           0.95     40618
   macro avg       0.96      0.95      0.95     40618
weighted avg       0.96      0.95      0.95     40618



## Without smote

In [54]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(50)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100)               34800     
_________________________________________________________________
dense_26 (Dense)             (None, 32)                3232      
_________________________________________________________________
dropout_27 (Dropout)         (None, 32)              

In [55]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/5
656/656 [==============================] - 33s 50ms/step - loss: 0.0382 - accuracy: 0.9788 - precision: 0.9788 - recall: 0.9788 - auc: 0.9959 - val_loss: 0.0032 - val_accuracy: 0.9996 - val_precision: 0.9996 - val_recall: 0.9996 - val_auc: 0.9998
Epoch 2/5
656/656 [==============================] - 33s 51ms/step - loss: 0.0012 - accuracy: 0.9997 - precision: 0.9997 - recall: 0.9997 - auc: 1.0000 - val_loss: 9.3934e-05 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000
Epoch 3/5
656/656 [==============================] - 32s 49ms/step - loss: 4.3227e-04 - accuracy: 0.9999 - precision: 0.9999 - recall: 0.9999 - auc: 1.0000 - val_loss: 0.0044 - val_accuracy: 0.9995 - val_precision: 0.9995 - val_recall: 0.9995 - val_auc: 0.9996
Epoch 4/5
656/656 [==============================] - 33s 50ms/step - loss: 0.0061 - accuracy: 0.9989 - precision: 0.9989 - recall: 0.9989 - auc: 0.9996 - val_loss: 0.0182 - val_accuracy: 0.9975 - val_precision: 0.9975 - v

In [56]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))

[[20281    29]
 [    1   652]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20310
           1       0.96      1.00      0.98       653

    accuracy                           1.00     20963
   macro avg       0.98      1.00      0.99     20963
weighted avg       1.00      1.00      1.00     20963



# RNN

## With smote

In [88]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.SimpleRNN(30),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_17 (SimpleRNN)    (None, 30)                2850      
_________________________________________________________________
dense_52 (Dense)             (None, 32)                992       
_________________________________________________________________
dropout_52 (Dropout)         (None, 32)              

In [89]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 1
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

1270/1270 [==============================] - 66s 52ms/step - loss: 0.2486 - accuracy: 0.9157 - precision: 0.9157 - recall: 0.9157 - auc: 0.9685 - val_loss: 0.1902 - val_accuracy: 0.9356 - val_precision: 0.9356 - val_recall: 0.9356 - val_auc: 0.9756


In [90]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[19780   308]
 [ 2307 18223]]
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     20088
           1       0.98      0.89      0.93     20530

    accuracy                           0.94     40618
   macro avg       0.94      0.94      0.94     40618
weighted avg       0.94      0.94      0.94     40618



## Without smote

In [85]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.SimpleRNN(10),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_16 (SimpleRNN)    (None, 10)                750       
_________________________________________________________________
dense_50 (Dense)             (None, 32)                352       
_________________________________________________________________
dropout_50 (Dropout)         (None, 32)              

In [86]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 2
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/2
656/656 [==============================] - 35s 54ms/step - loss: 0.0919 - accuracy: 0.9776 - precision: 0.9776 - recall: 0.9776 - auc: 0.9868 - val_loss: 0.0399 - val_accuracy: 0.9925 - val_precision: 0.9925 - val_recall: 0.9925 - val_auc: 0.9955
Epoch 2/2
656/656 [==============================] - 34s 52ms/step - loss: 0.0330 - accuracy: 0.9930 - precision: 0.9930 - recall: 0.9930 - auc: 0.9972 - val_loss: 0.0381 - val_accuracy: 0.9907 - val_precision: 0.9907 - val_recall: 0.9907 - val_auc: 0.9975


In [87]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))

[[20188   122]
 [   74   579]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     20310
           1       0.83      0.89      0.86       653

    accuracy                           0.99     20963
   macro avg       0.91      0.94      0.93     20963
weighted avg       0.99      0.99      0.99     20963



# BIRNN

## With smote

In [68]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(30)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 60)                5700      
_________________________________________________________________
dense_38 (Dense)             (None, 32)                1952      
_________________________________________________________________
dropout_38 (Dropout)         (None, 32)              

In [69]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
1270/1270 [==============================] - 89s 70ms/step - loss: 0.2139 - accuracy: 0.9259 - precision: 0.9259 - recall: 0.9259 - auc: 0.9724 - val_loss: 0.1623 - val_accuracy: 0.9459 - val_precision: 0.9459 - val_recall: 0.9459 - val_auc: 0.9822
Epoch 2/5
1270/1270 [==============================] - 97s 76ms/step - loss: 0.1628 - accuracy: 0.9469 - precision: 0.9469 - recall: 0.9469 - auc: 0.9816 - val_loss: 0.1626 - val_accuracy: 0.9459 - val_precision: 0.9459 - val_recall: 0.9459 - val_auc: 0.9821
Epoch 3/5
1270/1270 [==============================] - 89s 70ms/step - loss: 0.1708 - accuracy: 0.9432 - precision: 0.9432 - recall: 0.9432 - auc: 0.9802 - val_loss: 0.1791 - val_accuracy: 0.9417 - val_precision: 0.9417 - val_recall: 0.9417 - val_auc: 0.9781
Epoch 4/5
1270/1270 [==============================] - 97s 76ms/step - loss: 0.1860 - accuracy: 0.9372 - precision: 0.9372 - recall: 0.9372 - auc: 0.9769 - val_loss: 0.1816 - val_accuracy: 0.9366 - val_precision: 0.9366 - v

In [70]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[19751   337]
 [ 2109 18421]]
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     20088
           1       0.98      0.90      0.94     20530

    accuracy                           0.94     40618
   macro avg       0.94      0.94      0.94     40618
weighted avg       0.94      0.94      0.94     40618



## Without smote

In [71]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(30)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 60)                5700      
_________________________________________________________________
dense_40 (Dense)             (None, 32)                1952      
_________________________________________________________________
dropout_40 (Dropout)         (None, 32)              

In [72]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/5
656/656 [==============================] - 50s 76ms/step - loss: 0.0627 - accuracy: 0.9707 - precision: 0.9707 - recall: 0.9707 - auc: 0.9916 - val_loss: 0.0063 - val_accuracy: 0.9992 - val_precision: 0.9992 - val_recall: 0.9992 - val_auc: 0.9993
Epoch 2/5
656/656 [==============================] - 45s 69ms/step - loss: 0.0047 - accuracy: 0.9991 - precision: 0.9991 - recall: 0.9991 - auc: 0.9997 - val_loss: 0.0044 - val_accuracy: 0.9993 - val_precision: 0.9993 - val_recall: 0.9993 - val_auc: 0.9996
Epoch 3/5
656/656 [==============================] - 46s 70ms/step - loss: 0.0048 - accuracy: 0.9991 - precision: 0.9991 - recall: 0.9991 - auc: 0.9997 - val_loss: 0.0074 - val_accuracy: 0.9993 - val_precision: 0.9993 - val_recall: 0.9993 - val_auc: 0.9995
Epoch 4/5
656/656 [==============================] - 50s 76ms/step - loss: 0.0053 - accuracy: 0.9992 - precision: 0.9992 - recall: 0.9992 - auc: 0.9996 - val_loss: 0.0113 - val_accuracy: 0.9985 - val_precision: 0.9985 - val_recal

In [73]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))

[[20309     1]
 [   12   641]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20310
           1       1.00      0.98      0.99       653

    accuracy                           1.00     20963
   macro avg       1.00      0.99      0.99     20963
weighted avg       1.00      1.00      1.00     20963



# CNN

## With smote

In [105]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Conv1D(16, 3, padding="same"))
model.add(tf.keras.layers.Conv1D(32, 3, padding="same"))
model.add(tf.keras.layers.Conv1D(64, 3, padding="same"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu')) 
#  model.add(keras.layers.Dense(64,  activation='relu')) 

model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 46, 16)            4816      
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 46, 32)            1568      
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 46, 64)            6208      
_________________________________________________________________
flatten_7 (Flatten)          (None, 2944)              0         
_________________________________________________________________
dense_68 (Dense)             (None, 128)               376960    
_________________________________________________________________
dense_69 (Dense)             (None, 2)               

In [106]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
1270/1270 [==============================] - 30s 24ms/step - loss: 0.1856 - accuracy: 0.9278 - val_loss: 0.1197 - val_accuracy: 0.9598
Epoch 2/5
1270/1270 [==============================] - 29s 23ms/step - loss: 0.0912 - accuracy: 0.9681 - val_loss: 0.1178 - val_accuracy: 0.9624
Epoch 3/5
1270/1270 [==============================] - 32s 25ms/step - loss: 0.0625 - accuracy: 0.9764 - val_loss: 0.1253 - val_accuracy: 0.9598
Epoch 4/5
1270/1270 [==============================] - 30s 24ms/step - loss: 0.0481 - accuracy: 0.9808 - val_loss: 0.1589 - val_accuracy: 0.9561
Epoch 5/5
1270/1270 [==============================] - 30s 24ms/step - loss: 0.0349 - accuracy: 0.9858 - val_loss: 0.1802 - val_accuracy: 0.9616


In [107]:
predicted_prob = model.predict(X_test)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))

[[19759   329]
 [ 1229 19301]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     20088
           1       0.98      0.94      0.96     20530

    accuracy                           0.96     40618
   macro avg       0.96      0.96      0.96     40618
weighted avg       0.96      0.96      0.96     40618



## Without smote

In [102]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Conv1D(16, 3, padding="same"))
model.add(tf.keras.layers.Conv1D(32, 3, padding="same"))
model.add(tf.keras.layers.Conv1D(64, 3, padding="same"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu')) 
#  model.add(keras.layers.Dense(64,  activation='relu')) 

model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 46, 100)           2804800   
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 46, 16)            4816      
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 46, 32)            1568      
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 46, 64)            6208      
_________________________________________________________________
flatten_6 (Flatten)          (None, 2944)              0         
_________________________________________________________________
dense_66 (Dense)             (None, 128)               376960    
_________________________________________________________________
dense_67 (Dense)             (None, 2)               

In [103]:
num_epochs = 5
history = model.fit(
    X_train1,
    Training_labels1,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test1, Testing_labels1)
)

Epoch 1/5
656/656 [==============================] - 17s 25ms/step - loss: 0.0425 - accuracy: 0.9899 - val_loss: 9.7999e-04 - val_accuracy: 0.9998
Epoch 2/5
656/656 [==============================] - 16s 24ms/step - loss: 5.8253e-05 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9997
Epoch 3/5
656/656 [==============================] - 15s 23ms/step - loss: 1.5667e-05 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 4/5
656/656 [==============================] - 17s 25ms/step - loss: 8.5605e-07 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 5/5
656/656 [==============================] - 15s 23ms/step - loss: 5.0170e-07 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 0.9997


In [104]:
predicted_prob = model.predict(X_test1)
predicted = [np.argmax(pred) for pred in predicted_prob]
print(confusion_matrix(Y_test1,predicted))
print(classification_report(Y_test1,predicted))

[[20310     0]
 [    6   647]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20310
           1       1.00      0.99      1.00       653

    accuracy                           1.00     20963
   macro avg       1.00      1.00      1.00     20963
weighted avg       1.00      1.00      1.00     20963

